In [1]:
# generate training data
import requests
import json
import os
from dotenv import load_dotenv
load_dotenv()
openai_key = os.getenv('openaikey')

if openai_key:
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer "+openai_key,
    }
    print('headers loaded')

headers loaded


In [2]:
# Brainstorm intentions and focuses with ChatGPT
## > We are coming up with themes for mindfulness meditations. 
##   Generate a list of topics or feelings that might lead someone to want to do a mindful meditation.
# ...cultivated list...
## > Ok, now for each of these intentions, come up with 5 focuses:
## > Great, now, convert this to a python dictionary, with the intentions as keys and the focuses as a value list. 
##   Keep everything lowercase and remove periods.
intentions = {
    "reduce stress": [
        "focusing on deep, rhythmic breathing",
        "visualization of a calm and peaceful place",
        "mindful awareness of bodily sensations to release tension",
        "guided imagery to foster relaxation",
        "progressive muscle relaxation, from toes to head"
    ],
    "balance emotions": [
        "observing emotions without judgment",
        "breathing exercises to stabilize mood swings",
        "visualization of emotional equilibrium",
        "mindful acknowledgment of positive and negative feelings",
        "cultivating self-compassion and understanding"
    ],
    "improve focus": [
        "concentration on a single object or sound",
        "mindful listening to ambient sounds",
        "body scan meditation for enhanced awareness",
        "visualization of mental clarity and sharpness",
        "practicing open awareness meditation"
    ],
    "manage anxiety": [
        "deep breathing techniques for calming",
        "acknowledging anxious thoughts without attachment",
        "visualization of releasing worries and fears",
        "grounding exercises, focusing on the present moment",
        "guided imagery for inner peace"
    ],
    "enhance self-awareness": [
        "reflecting on personal values and beliefs",
        "observing thoughts without judgment",
        "body scan to understand physical cues and sensations",
        "contemplating personal growth and changes over time",
        "meditation on self-identity and purpose"
    ],
    "improve sleep": [
        "progressive muscle relaxation for physical ease",
        "deep breathing for slowing down the mind",
        "visualization of drifting into peaceful sleep",
        "guided imagery of a tranquil night",
        "mindful review of the day for closure"
    ],
    "manage anger": [
        "acknowledging anger without acting on it",
        "deep breathing to cool down heated emotions",
        "visualization of transforming anger into understanding",
        "reflecting on the root causes of anger",
        "practicing forgiveness and letting go"
    ],
    "develop compassion": [
        "loving-kindness meditation for others and self",
        "visualization of empathy and understanding",
        "reflecting on common human experiences",
        "cultivating a non-judgmental attitude",
        "practicing gratitude and appreciation"
    ],
    "cope with grief": [
        "acknowledging feelings of loss and sadness",
        "deep breathing for emotional release",
        "visualization of cherished memories",
        "mindful walking to process grief",
        "reflecting on the cycle of life and acceptance"
    ],
    "increase mindfulness": [
        "practicing present moment awareness",
        "mindful eating to savor each bite",
        "observing the natural world with curiosity",
        "mindful movement or walking meditation",
        "daily gratitude reflection"
    ],
    "expand creativity": [
        "visualization of creative energy flow",
        "free-flowing thought observation",
        "mindful engagement with art or music",
        "breathing exercises to open the mind",
        "reflecting on inspiring themes or ideas"
    ],
    "build resilience": [
        "reflecting on past challenges and growth",
        "visualization of inner strength and adaptability",
        "breathing through difficulties",
        "cultivating a positive outlook",
        "grounding exercises in the face of adversity"
    ],
    "reduce negative thoughts": [
        "recognizing and labeling negative thoughts",
        "deep breathing to create mental space",
        "visualization of letting go of negativity",
        "mindful redirection of focus to positivity",
        "practicing gratitude to counter negative biases"
    ],
    "cultivate gratitude": [
        "daily mental acknowledgment of things to be grateful for",
        "reflecting on positive life aspects",
        "deep breathing with a focus on thankfulness",
        "visualization of life's gifts",
        "acknowledging acts of kindness received and given"
    ],
    "boost self-esteem": [
        "affirmations of self-worth and capability",
        "visualization of personal achievements",
        "reflecting on unique strengths and qualities",
        "mindful acceptance of compliments",
        "body positivity exercises"
    ],
    "overcome perfectionism": [
        "acknowledging imperfections with kindness",
        "visualization of realistic goals and expectations",
        "mindful acceptance of mistakes as learning opportunities",
        "reflecting on the value of progress over perfection",
        "deep breathing to let go of self-criticism"
    ],
    # a blank one to get some generic meditations
    "":[None,None,None,None,None]
}

In [3]:
def get_prompt(objective, focus = None):
    intention = '' if objective == '' else ' to '+objective
    if focus:
        intention += ' with a focus on '+focus
    return f"You are a secular buddhist monk. Give me a daily meditation{intention}. Do not include any introductory text."

def get_meditation(json_data):
    response = requests.post(
        "https://api.openai.com/v1/chat/completions",
        headers=headers,
        json=json_data,
    )
    result = response.json()
    meditation = None
    if result.get('choices'):
        cost = result['usage']['prompt_tokens'] * 1e-6 + result['usage']['completion_tokens'] * 2e-6
        meditation = result['choices'][0]['message']['content']
    return meditation, cost

In [4]:
# test
oo = 'boost self-esteem'
json_data = {
    "model": 'gpt-3.5-turbo',
    "messages": [{"role": "user", "content": get_prompt(oo)}],
    "temperature": 1.1,
}
get_meditation(json_data)

('Sit comfortably, close your eyes, and take a deep breath in, filling your lungs completely. As you exhale, release any tension or negativity you may be holding onto.\n\nNow, bring your attention to your breath, allowing each inhale and exhale to flow naturally. Be fully present in this moment.\n\nShift your focus inward and bring forth thoughts of self-compassion and kindness. Visualize a light within you that represents your inherent worth and unique strengths.\n\nWith each breath, imagine this light growing brighter and spreading to every part of your being. Feel the warmth and positivity flowing throughout your body.\n\nAs you continue to breathe, choose a positive affirmation that resonates with you. It could be a simple phrase like, "I am enough" or "I embrace my worth." Repeat this affirmation silently or out loud, emphasizing each word with genuine belief.\n\nReflect on your accomplishments, big or small, and the challenges you have overcome. Remind yourself of your resilience

In [8]:
results = {}
nper = 3 # number of meditations per intention
cost = 0 # cost tally

for objective, ff in intentions.items():
    print('\n'+objective)
    label_objective = 'general' if objective == '' else objective
    # initialize
    if label_objective not in results:
        results[label_objective] = []
    for focus in ff:
        #print(focus+' ~ ')
        json_data = {
            "model": 'gpt-3.5-turbo',
            "messages": [{"role": "user", "content": get_prompt(objective, focus)}],
            "temperature": 1.1,
        }
        for ii in range(nper):
            meditation, runcost = get_meditation(json_data)
            cost += runcost
            results[label_objective].append(meditation)
            print("*",end="")
print("\ntotal cost $",str(round(cost,2)))


reduce stress
***************
balance emotions
***************
improve focus
***************
manage anxiety
***************
enhance self-awareness
***************
improve sleep
***************
manage anger
***************
develop compassion
***************
cope with grief
***************
increase mindfulness
***************
expand creativity
***************
build resilience
***************
reduce negative thoughts
***************
cultivate gratitude
***************
boost self-esteem
***************
overcome perfectionism
***************

***************
total cost $ 0.21


In [31]:
import re

# Define a function to convert numbers to spoken words
def convert_numbers_to_spoken_words(match):
    number = match.group(0)
    spoken_words = {
        '1. ': 'First, ',
        '2. ': 'Next, ',
        '3. ': 'Then, ',
        '4. ': '',
        '5. ': '',
        '6. ': '',
        '7. ': '',
        '8. ': '',
        '9. ': '',
        '10. ': '', 
        '11. ': '', 
        '12. ': '', 
        '13. ': '', 
        '14. ': '', 
        '15. ': '',  # You can extend this for more numbers if needed
    }
    return spoken_words.get(number, number)

def cleanup_meditation(input_text):
    # Use re.sub to replace the matched text with an empty string
    result_text = re.sub(r'^Daily .*\n', '', input_text)
    # Use regular expressions to find and replace numbers with spoken words
    converted_text = re.sub(r'\d+\.\s', convert_numbers_to_spoken_words, result_text)
    return converted_text.lstrip('\n')

print(cleanup_meditation(results['balance emotions'][2]))

First, Find a quiet and comfortable spot where you can sit or lie down. Close your eyes gently if it feels comfortable, allowing yourself to relax and settle into a comfortable posture.

Next, Take a few deep, soothing breaths, letting go of any tension or stress you may be feeling. Allow the breath to become your anchor, bringing your focus to the present moment.

Then, Begin by observing the sensations in your body. Bring your awareness to the physical sensations that arise as you breathe in and out. Notice the subtle movements, temperature, and any areas of tension or relaxation.

Gradually shift your attention towards your emotions. Without judgment or trying to change anything, simply observe how your emotions manifest in the present moment. It might be a feeling of joy, sadness, anger, fear, or any other emotion. Pay attention to its presence, intensity, and how it affects different areas of your body.

Allow the emotions to be just as they are, without trying to suppress or hold

In [25]:
# write json for importing into huggingface dataset
import json

ds_out = []
for t in results:
    for m in results[t]:
        ds_out.append({'theme':t,'meditation':cleanup_meditation(m)})
with open('meditation-dataset.json', 'w') as fp:
    json.dump(ds_out, fp)

# now upload to huggingface

In [26]:
import re
nsentences = []
for t in results:
    for m in results[t]:
        mt = re.sub(r'[1-9]\.|[1-9] \.', '', m)
        nsentences.append(len(m.split(".")))

In [27]:
sum(nsentences)/len(nsentences) # average number of sentences per meditation

24.50588235294118

In [32]:
from datasets import load_dataset
dataset = load_dataset("berdaniera/meditation", split="train")
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /Users/berdaniera/.cache/huggingface/datasets/berdaniera___json/berdaniera--meditation-4a72f7da580b0746/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


Dataset({
    features: ['theme', 'meditation'],
    num_rows: 255
})